In [1]:
import pandas as pd

In [6]:
# Reading in filtered EPA data to NYC specifically 
EPA_NYC_df = pd.read_csv('github_data/EPA_NYC_out.csv')
EPA_NYC_df.head()

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,148837,3.608100e+11,3.608100e+11,36,81,25900,2,408.0,"New York-Newark, NY-NJ-CT-PA",35620.0,...,0.506986,0.000222,0.528185,19,15,17,18,17.333333,1016.744396,64619.12044
1,148838,3.608100e+11,3.608100e+11,36,81,16600,2,408.0,"New York-Newark, NY-NJ-CT-PA",35620.0,...,0.029180,0.000061,0.144270,9,9,15,19,14.333333,1485.032311,91377.23524
2,148839,3.608110e+11,3.608110e+11,36,81,122702,1,408.0,"New York-Newark, NY-NJ-CT-PA",35620.0,...,0.154780,0.000067,0.159039,1,1,10,18,9.666667,1110.284578,72542.55913
3,148840,3.608100e+11,3.608100e+11,36,81,28300,5,408.0,"New York-Newark, NY-NJ-CT-PA",35620.0,...,0.535719,0.000238,0.567026,5,20,1,19,10.833333,1096.360269,36881.79980
4,148841,3.608110e+11,3.608110e+11,36,81,148300,4,408.0,"New York-Newark, NY-NJ-CT-PA",35620.0,...,0.082963,0.000036,0.084785,14,12,19,14,15.333333,2652.063831,286388.75040


Below, we have 2167 census tracts for NYC in the dataset (according to Google there are 2168, so this tracks)

In [3]:
unique_groups = EPA_NYC_df.groupby(['TRACTCE', 'COUNTYFP']).ngroups
print("Number of unique combinations:", unique_groups)

Number of unique combinations: 2167


## Next Step: Data visualizations and statistics
- 1: pick features we wanna examine (mainly from EPA dataset)
- 2: group by the 2167 census tracts
    - figure out how to merge certain data when grouping (mean, median, max)
- 3: regression, clustering, pca, correlations, etc.

In [4]:
# group by census tract

6493